# Data: Fairfax County Fire and Rescue Department call data and patient demographics 
Team: Fairfax Consultants

In [ ]:
print("Hello Fairfax Consultants, Welcome Aboard")
#pip install nbconvert #or conda install nbconvert  #to save the notebook as .pdf
#nbconvert[webpdf]
#pip install pyppeteer
#https://github.com/jgm/pandoc/releases/tag/2.18

In [ ]:
#Check the version
!jupyter --version
!python --version

In [ ]:
#Import libraries
import os

import pandas as pd
#pd.set_option('display.max_rows', None)

import numpy as np
import math
#Import Viz libraries


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Altair can be installed, along with the example datasets in vega_datasets, using:$ pip install altair vega_datasets
#https://altair-viz.github.io/gallery/index.html

import altair as alt #pip install altair (from anaconda Powershell)
from vega_datasets import data # pip install vega_datasets/ #Only to use data from vegas data sets or else can be ignored

#To avoid MaxRowsError: in big data sets with altair
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')
#pip install altair_data_server
#alt.data_transformers.enable('data_server')
# Optional in Jupyter Notebook: requires an up-to-date vega nbextension.
#alt.renderers.enable('notebook') #$ pip install vega #https://altair-viz.github.io/user_guide/display_frontends.html

#pip install altair_viewer
#alt.renderers.enable('altair_viewer') 
#pip install altair_saver #to save interactive charts with interactive elements

#alt.renderers.enable('svg') #renders the chart as a static svg image within a Jupyter notebook.
#chart.save('chart.html', embed_options={'renderer':'svg'}) #o change to svg rendering, use the embed_options as such

In [ ]:
#Import all sheet by index
# # Set current working directory
# os.chdir("C:\\Users\\matth\\OneDrive\\Documents\\GMU\\DAEN 690\\Sponsor Files")
#df = pd.read_excel("E:\DAEN_690\Data\RawData_GMU_Summer2022.xlsx", sheet_name = None)#[0,1,2,3])

df = pd.read_excel("E:\DAEN_690\Data\RawData_GMU_Summer2022_SecondaryImpression.xlsx", sheet_name = None)#[0,1,2,3])

# # reading csv file and at a same time using converters attribute which will remove extra space
# df = pd.read_csv('\\student_data.csv', converters={'Name': str.strip(),
#                                                 'Blood Group' : str.strip(),
#                                                 'Gender' : str.strip() } )

#df = pd.read_excel("E:\DAEN_690\Data\Copy of RawData_GMU_Summer2022_SecondaryImpression.xlsx", sheet_name = None)#[0,1,2,3]) #in laptop
#df = pd.read_excel("E:\DAEN_690\Data\Copy of RawData_GMU_Summer2022.xlsx", sheet_name = None)#[0,1,2,3]) #in laptop
#pd.ExcelFile()
print(df)

In [ ]:
#Imported as Dictionary, view keys or values
print(df.keys())
print(df.values())
print(df['Patient Demographics']) #view 

In [ ]:
#convert Dicationary to Data Frames
PatDemo = pd.DataFrame(df['Patient Demographics'])
SEPSIS = pd.DataFrame(df['sepsis'])
STEMI = pd.DataFrame(df['STEMI'])
STROKE = pd.DataFrame(df['stroke'])

#View Headings
print("\n PatDemo:\n",PatDemo.head(1))
print("\n SEPSIS:\n",SEPSIS.head(1))
print("\n STEMI:\n",STEMI.head(1))
print("\n STROKE:\n",STROKE.head(1))

In [ ]:
#View Columns for each data frame

print("\n PatDemo:\n",PatDemo.columns)
print("\n SEPSIS:\n",SEPSIS.columns )
print("\n STEMI:\n",STEMI.columns )
print("\n STROKE:\n",STROKE.columns)

In [ ]:
#Count Nulls
print(PatDemo.isnull().sum())

In [ ]:
#With the help of heatmap, we can see the amount of data that is missing from the attribute
sns.heatmap(PatDemo.isnull(),cbar=False,cmap='viridis')

In [ ]:
#Check if any columns or values are duplicated.

print("\n If any column header redundant?:",PatDemo.columns.duplicated())
print("\n Sex & Gender Duplicated?   : ",PatDemo['PatientSex'].equals(PatDemo['PatientGender']))
print("\n Race Duplicated?           : ",PatDemo['Patient_Race_List_Raw'].equals(PatDemo['PatientRace']))
print("\n Gender and GRaw Duplicated?: ",PatDemo['Patient_Gender_Raw'].equals(PatDemo['PatientGender']))

#PatDemo.columns
#even if logicaltest says not duplicated, it is okay to delete 'PatientSex',  'Patient_Gender_Raw', 'Patient_Race_List_Raw'

In [ ]:
#drop reduntant columns
PatDemo.drop(["Patient_Race_List_Raw", "Patient_Gender_Raw",'PatientGender'], axis =1,  inplace =True)
PatDemo.head(2)

In [ ]:
#rename patient demographic column 'Patient_ID_Internal' to 'PtIDInternal'
PatDemo.rename(columns = {'Patient_ID_Internal':'PtIDInternal'}, inplace = True)

In [ ]:
#Count of unique values in each rows for each column
  
print("No.of.unique values in each column :\n", PatDemo.nunique(axis=0))

In [ ]:
#read unique values, how many patients under particular age group

#print("\n unique values: ", PatDemo['AgeGroup_HIPAA'].unique() )
#print("\n Count of unique values: ", len(PatDemo['AgeGroup_HIPAA'].unique()))
#print("\n Count of unique values: ", PatDemo.AgeGroup_HIPAA.nunique())
#print("\n Count of each unique values: ", list(PatDemo.AgeGroup_HIPAA.value_counts()))
print("\n Count of each unique values: ", PatDemo.groupby('AgeGroup_HIPAA').PtIDInternal.nunique())

In [ ]:
#Convert AgeGroup_HIPAA to string to get rid of datetime format value
PatDemo['AgeGroup_HIPAA']= PatDemo['AgeGroup_HIPAA'].astype('str')
#print("\n sorted age group:",sorted(PatDemo['AgeGroup_HIPAA'].unique()))
print("\n Count of each unique values: ", PatDemo.groupby('AgeGroup_HIPAA').PtIDInternal.nunique())

In [ ]:
#check the data types for rest of the columns
print(PatDemo.info())
print("\n details: \n",PatDemo['AgeGroup_HIPAA'].describe())
# PatDemo.dtypes
# PatDemo.index
# PatDemo.shape
# PatDemo.memory_usage


In [ ]:
#Replace '2022-10-19 00:00:00' with 10-19 and nan with Unk, my logic is even NULL is also unknown in a way

PatDemo["AgeGroup_HIPAA"] = PatDemo["AgeGroup_HIPAA"].replace({'2022-10-19 00:00:00':'10_19','20-29':'20_29','50-59':'50_59', 
                                                               '80-89':'80_89','70-79':'70_79','40-49':'40_49','Under 10':'<10', 
                                                               '60-69':'60_69','30-39':'30_39', 'nan':'Unk'})

#sorted(PatDemo["AgeGroup_HIPAA"].unique())
print("\n Count of each unique values: ", PatDemo.groupby('AgeGroup_HIPAA').PtIDInternal.nunique())
# Total=sum(PatDemo['AgeGroup_HIPAA'].value_counts(dropna=True))
# Total
# PatDemo.AgeGroup_HIPAA.value_counts(dropna=True)

In [ ]:
#Binarize the boolen values 
#PatDemo.columns
#Get the boolean values for Patients Sex
PatDemo = pd.get_dummies(PatDemo, columns=['PatientRace', 'PatientEthnicity', 'PatientSex','AgeGroup_HIPAA']) #, drop_first=True) #Since thers only two values male or female.
PatDemo

In [ ]:
#Lets drop redundant columns:
PatDemo.drop(['PatientSex_Male','IsJuvenileUnder18','IsSenior65+'],axis =1,  inplace =True)
#rename patient demographic column 'Patient_ID_Internal' to 'PtIDInternal'
PatDemo.rename(columns = {'PatientSex_Female':'PatientSex_FM'}, inplace = True)
PatDemo

#SEPSIS_Data Cleaning and Merge with Patients Demographics

In [ ]:
#View Header and count null

print("\n header:\n",SEPSIS.head(2))

print("\n count null before merge:\n", SEPSIS.isnull().sum())


In [ ]:
#Check if any columns or values are duplicated.
#SEPSIS['EmergencyDepartmentDiagnosis'].unique()

print("\n If any column header redundant?:",SEPSIS.columns.duplicated())

print("\n Emergency & Hospital have same values?   : ",SEPSIS['EmergencyDepartmentDiagnosis'].equals(SEPSIS['HospitalDiagnosis']))


print("\n Primary and Secondary Impression Duplicated?: ",SEPSIS['PrimaryImpression'].equals(SEPSIS['SecondaryImpression']))


# print("\n Race Duplicated?           : ",SEPSIS['Patient_Race_List_Raw'].equals(SEPSIS['PatientRace']))
# print("\n Gender and GRaw Duplicated?: ",SEPSIS['Patient_Gender_Raw'].equals(SEPSIS['PatientGender']))


In [ ]:
## create a new column that is a combination of the emergency deparment diagnosis & hospital diagnosis
# https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-pandas-dataframe
SEPSISclean = SEPSIS

#SEPSISclean["overall_diagnosis"] = SEPSISclean["EmergencyDepartmentDiagnosis"].astype(str).fillna('') + SEPSISclean["HospitalDiagnosis"].astype(str).fillna('')
SEPSISclean["overall_diagnosis"] = SEPSISclean["EmergencyDepartmentDiagnosis"].fillna('') + SEPSISclean["HospitalDiagnosis"].fillna('')

SEPSISclean["Merged_Impression"] = SEPSISclean["PrimaryImpression"].fillna('') + SEPSISclean["SecondaryImpression"].fillna('')

##drop reduntant columns
#SEPSIS.drop(["EmergencyDepartmentDiagnosis", "HospitalDiagnosis"], axis =1,  inplace =True)

##View Header and count null

print("\n header:\n",SEPSISclean.head(2))

print("\n count null after merge:\n", SEPSISclean.isna().sum())

#SEPSISclean.head(10)

In [ ]:
#SEPSISclean.dtypes
print("\n count NaN after merge:\n",SEPSISclean[SEPSISclean == ''].count())
#SEPSISclean['overall_diagnosis'].eq('nannan').sum()##df1[['overall_diagnosis ', 'col3', 'col4']].eq(1).sum()

In [ ]:
SEPSISclean["overall_diagnosis"].unique()

# Infectious -Sepsis (A41.9)
# "Sepsis, unspecified organism (A41.9)"
#"Other specified sepsis (A41.89)"
# "Severe sepsis with septic shock (R65.21), 
# "Severe sepsis without septic shock (R65.20)"


In [ ]:
## create a new column that is 1 if the overall_diagnosis column contains the string "A41.9", 0 if not

SEPSISclean["sepsis_outcome"] = pd.np.where(SEPSISclean["overall_diagnosis"].str.contains("Sepsis", na=False, case=False), 1, 
                                #pd.np.where(SEPSISclean["overall_diagnosis"].str.contains("sepsis", na=False), "1",
                                pd.np.where(SEPSISclean["overall_diagnosis"].str.contains("A41.9", na=False, case=False), 1, 0))#)
# sepsis_demo["sepsis_outcome"] = np.where(sepsis_demo["overall_diagnosis"].str.contains("A41.9", na=False, case=False), "1",
#                                 np.where(sepsis_demo["overall_diagnosis"].str.contains("A41.89", na=False, case=False), "1", "0"))


SEPSISclean.head()
#binarize primary Impr
SEPSISclean["ProviderImpression"] = pd.np.where(SEPSISclean["Merged_Impression"].str.contains("Sepsis", na=False, case=False), 1, 
                               
                             pd.np.where(SEPSISclean["Merged_Impression"].str.contains("A41.9", na=False, case=False), 1, 0))
SEPSISclean.head()
#Accuracy
SEPSISclean["Accuracy"] = pd.np.where(SEPSISclean["ProviderImpression"] == SEPSISclean["sepsis_outcome"], 1, 0) #Match -1, MisMatch 0
SEPSISclean.head()

In [ ]:
#To find out  if it is Under or Over Triage.
#TRIAGE is the sorting of and allocation of treatment to patients

SEPSISclean["Triage0"] = np.where(((SEPSISclean["ProviderImpression"] == SEPSISclean["sepsis_outcome"])),'Triage','')
SEPSISclean["Triage1"] = np.where(((SEPSISclean["ProviderImpression"] == 1) &  (SEPSISclean["sepsis_outcome"] == 0)),'OverTriage','')
SEPSISclean["Triage2"] = np.where(((SEPSISclean["ProviderImpression"] == 0) &  (SEPSISclean["sepsis_outcome"] == 1)),'UnderTriage','')
SEPSISclean["Triage"] = SEPSISclean["Triage0"].fillna('') + SEPSISclean["Triage1"].fillna('') + SEPSISclean["Triage2"].fillna('')

print('Head: ',SEPSISclean.head())
print('UniqueValue: ',SEPSISclean["Triage"].unique())

In [ ]:
#Check for unique values in each columns

print("\n Merged_Impressions Values: \n", SEPSISclean['Merged_Impression'].unique())
print("\n ProviderImpression Values: \n", SEPSISclean['ProviderImpression'].unique())
print("\n Unique qSOFA values: \n",SEPSISclean['qSOFA'].unique())
print("\n Unique Sepsis Statusvalues: \n",SEPSISclean['SepsisStatus'].unique())

In [ ]:
#Crete dummies for SepsisStatus
SEPSISclean = pd.get_dummies(SEPSISclean, columns=['SepsisStatus'])#, drop_first=True)
SEPSISclean

In [ ]:
#To iter through the values/ categories list nan are replaced with Unknown

SEPSISclean["qSOFA"] = SEPSISclean["qSOFA"].fillna('Unknown')
SEPSISclean['qSOFA'].unique()



#Create dummies for qSOFA

# SEPSISclean = pd.get_dummies(SEPSISclean, columns=['qSOFA'])#, drop_first=True) #will create 7 combinations, in fact we have only three criteria.
# SEPSISclean
#following method will identify three condition within the strings.

dummies = SEPSISclean['qSOFA'].str.get_dummies(sep=', ') #(SEPSISclean[variable], prefix = variable)   # Get the dummy variables from pandas
SEPSISclean = pd.concat([SEPSISclean,dummies],axis=1)       # concat dummy variables into original data 
#SEPSISclean.drop(SEPSISclean['qSOFA'],axis=1,inplace=True)      # inplace  = True : becasue we want to work on the dataframe directly 
SEPSISclean


In [ ]:
##Drop redundant column 
SEPSISclean.drop(['MonthYear','PrimaryImpression','SecondaryImpression',"EmergencyDepartmentDiagnosis","HospitalDiagnosis",
                  "overall_diagnosis",'qSOFA','Merged_Impression','Triage1','Triage2','Triage0','Unknown'],
                 axis =1,  inplace =True)
SEPSISclean.head()

Merge with Patients DemoGraphic Informations

In [ ]:
# merging the files how='inner' by default.
PdmSEPSIS = pd.merge(PatDemo,SEPSISclean, on='PtIDInternal') 
print(PdmSEPSIS.head(3))

#Rearrange the columns
last_cols = ['Accuracy', 'Triage']
first_cols = [col for col in PdmSEPSIS.columns if col not in last_cols]
PdmSEPSIS = PdmSEPSIS[first_cols+last_cols]
print(PdmSEPSIS.head(3))

# #Write as .csv
#PdmSEPSIS.to_excel("E:\DAEN_690\Data\PdmSEPSIS.xlsx",sheet_name='SEPSIS' , index=False)
PdmSEPSIS.to_csv("E:\DAEN_690\Data\PdmSEPSIS1.csv",index=False)

In [ ]:
#Lets drop irrelavant  columns:
PdmSEPSIS.drop(['PtIDInternal'],axis =1,  inplace =True)
print(PdmSEPSIS.head(3))
print("\n info: ", PdmSEPSIS.info())

#STEMI_Data Cleaning and Merge with Patients Demographics
Repeat the same process for STEMI

In [ ]:
#View Header and count null

print("\n header:\n",STEMI.head(2))

print("\n count null before merge:\n", STEMI.isnull().sum())

In [ ]:
#Check if any columns or values are duplicated.
#STEMI['EmergencyDepartmentDiagnosis'].unique()
print("\n If any column header redundant?:",STEMI.columns.duplicated())

print("\n Emergency & Hospital have same values?   : ",STEMI['EmergencyDepartmentDiagnosis'].equals(STEMI['HospitalDiagnosis']))

print("\n Primary and Secondary Impression Duplicated?: ",STEMI['PrimaryImpression'].equals(STEMI['SecondaryImpression']))

# print("\n Race Duplicated?           : ",STEMI['Patient_Race_List_Raw'].equals(STEMI['PatientRace']))
# print("\n Gender and GRaw Duplicated?: ",STEMI['Patient_Gender_Raw'].equals(STEMI['PatientGender']))

In [ ]:
## create a new column that is a combination of the emergency deparment diagnosis & hospital diagnosis
# https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-pandas-dataframe
STEMIclean = STEMI

#STEMIclean["overall_diagnosis"] = STEMIclean["EmergencyDepartmentDiagnosis"].astype(str) + STEMIclean["HospitalDiagnosis"].astype(str)
STEMIclean["overall_diagnosis"] = STEMIclean["EmergencyDepartmentDiagnosis"].fillna('') + STEMIclean["HospitalDiagnosis"].fillna('')
STEMIclean["Merged_Impression"] = STEMIclean["PrimaryImpression"].fillna('') + STEMIclean["SecondaryImpression"].fillna('')

##drop reduntant columns
#STEMI.drop(["EmergencyDepartmentDiagnosis", "HospitalDiagnosis"], axis =1,  inplace =True)

##View Header and count null

print("\n header:\n",STEMIclean.head(2))

print("\n count null after merge:\n", STEMIclean.isna().sum())

In [ ]:
#STEMIclean.dtypes
#print("\n count NaN after merge:\n",STEMIclean[STEMIclean == 'nannan'].count()) #975
print("\n count NaN after merge:\n",STEMIclean[STEMIclean == ''].count()) #975
#STEMIclean['overall_diagnosis'].eq('nannan').sum()##df1[['overall_diagnosis ', 'col3', 'col4']].eq(1).sum()

In [ ]:
STEMIclean.head(10)

In [ ]:
STEMIclean["Merged_Impression"].unique()

# CV - Myocardial Infarction (Non-STEMI), With or Without Chest Pain (I21.4) are excluded
# CV - STEMI of Anterior Wall, With or Without Chest Pain (I21.0)
# CV - STEMI of Inferior Wall, With or Without Chest Pain (I21.1)
# CV - STEMI of other sites, With or Without Chest Pain (I21.2)


In [ ]:
## create a new column that is 1 if the overall_diagnosis column contains the string "A41.9", 0 if not
#I21.0, I21.1, I21.2 #Codes used for STEMI
STEMIclean["STEMI_outcome"] = pd.np.where(STEMIclean["overall_diagnosis"].str.contains("I21.0", na=False, case=False), 1, 0)

#pd.np.where(STEMIclean["overall_diagnosis"].str.contains("(STEMI)", na=False, case=False), "1", 
                                #pd.np.where(STEMIclean["overall_diagnosis"].str.contains("STEMI", na=False), "1",
                                #)#)
print(STEMIclean.head())
#Write as .csv
#STEMIclean.to_excel("E:\DAEN_690\Data\PdmSTEMI.xlsx",sheet_name='STEMI' , index=False)

In [ ]:
## create a new column that is STEMI if the PrimaryImpression column contains the string "STEMI", Other if not
STEMIclean["PrimaryImpr"] = pd.np.where(STEMIclean["Merged_Impression"].str.contains("CV - STEMI", na=False, case=False), "1",
                                        (pd.np.where(STEMIclean["Merged_Impression"].str.contains("I21.", na=False, case=False),"1","0")))   
                            
                            #pd.np.where(STEMIclean["PrimaryImpression"].str.contains("I21.0" or "I21.1" or "I21.2", na=False, case=False), "STEMI", "Other")   
                            
print(STEMIclean.head())  

#Write as .csv
STEMIclean.to_excel("E:\DAEN_690\Data\PdmSTEMI.xlsx",sheet_name='STEMI' , index=False)

In [ ]:
STEMIclean['DCPSAMI'] = STEMIclean['DoesClinicalPicSuggestAcuteMI'].replace(['No','Uncertain','Yes'],['NO ClinicalPicSuggestAcuteMi','ClinicalPicSuggest Uncertain about AcuteMi','ClinicalPicSuggestAcuteMi'])
    
STEMIclean['DPIstSEl'] = STEMIclean['DoesProviderInterpretstSegmentElevation'].replace(['No','Uncertain','Yes'],['NO stSegmentElevation','Uncertain stSegmentElevation','stSegmentElevation'])
    
STEMIclean['DMSAMI'] = STEMIclean['DoesMonitorStateAcuteMI'].replace(['No','Uncertain','Yes'],['NO MonitorStateAcuteMi','MonitorState Uncertain about AcuteMi','MonitorStateAcuteMi',])

In [ ]:
#Merge Questionnaire columns to easily analyze/ pivot
STEMIclean["AcuteMI_stSegmentElevation"] = STEMIclean['DCPSAMI'].fillna('') + STEMIclean['DPIstSEl'].fillna('') + STEMIclean['DMSAMI'].fillna('')
#Write as .csv
#STEMIclean.to_excel("E:\DAEN_690\Data\PdmSTEMI.xlsx",sheet_name='STEMI' , index=False)    

In [ ]:
print("\n PImpressions Values: \n", STEMIclean['PrimaryImpression'].unique())
print("\n PImpr Values: \n", STEMIclean['PrimaryImpr'].unique())
#print("\n Unique qSOFA values: \n",SEPSISclean['qSOFA'].unique())

In [ ]:
STEMIclean
#Write as .xlsx
STEMIclean.to_excel("E:\DAEN_690\Data\STEMIclean.xlsx",sheet_name='STEMI' , index=False)   

In [ ]:
STEMIclean.columns

In [ ]:
##Drop redundant column 
STEMIclean.drop(['MonthYear','PrimaryImpression','SecondaryImpression',"EmergencyDepartmentDiagnosis","HospitalDiagnosis","overall_diagnosis", 'DCPSAMI', 'DPIstSEl', 'DMSAMI'], axis =1,  inplace =True)
STEMIclean.head()

Merge with Patient Demographic Information

In [ ]:
# merging the files how='inner' by default.
PdmSTEMI = pd.merge(PatDemo,STEMIclean, on='PtIDInternal') 
print(PdmSTEMI.head())
#Write as .csv
PdmSTEMI.to_excel("E:\DAEN_690\Data\PdmSTEMI.xlsx",sheet_name='STEMI' , index=False)

In [ ]:
print(PdmSTEMI.head())
print(PdmSTEMI.columns)

In [ ]:
import altair as alt
#from vega_datasets import data

source = PdmSTEMI

PatSTEMIChart = alt.Chart(source).mark_bar().encode(
    x='AgeGroup_HIPAA:O',
    y='count(AgeGroup_HIPAA):Q',
    color='PrimaryImpr:N',
    column='STEMIstatus:N',
    tooltip=['PrimaryImpr', 'STEMI_outcome', 'AgeGroup_HIPAA:Q', 'count(AgeGroup_HIPAA)']
).interactive()
PatSTEMIChart .display()
#PatDemChart.save('PatDemChart.html')

#STEMI_Data Cleaning and Merge with Patients Demographics

REPEAT Process for STROKE

In [ ]:
#View Header and count null

print("\n header:\n",STROKE.head(2))

print("\n count null before merge:\n", STROKE.isnull().sum())

In [ ]:
#Check if any columns or values are duplicated.
#STROKE['EmergencyDepartmentDiagnosis'].unique()
print("\n If any column header redundant?:",STROKE.columns.duplicated())

print("\n Emergency & Hospital have same values?   : ",STROKE['EmergencyDepartmentDiagnosis'].equals(STROKE['HospitalDiagnosis']))
print("\n Primary and Secondary Impression Duplicated?: ",STROKE['PrimaryImpression'].equals(STROKE['SecondaryImpression']))
# print("\n Race Duplicated?           : ",STROKE['Patient_Race_List_Raw'].equals(STROKE['PatientRace']))
# print("\n Gender and GRaw Duplicated?: ",STROKE['Patient_Gender_Raw'].equals(STROKE['PatientGender']))

In [ ]:
## create a new column that is a combination of the emergency deparment diagnosis & hospital diagnosis
# https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-pandas-dataframe
STROKEclean = STROKE

STROKEclean["overall_diagnosis"] = STROKEclean["EmergencyDepartmentDiagnosis"].astype(str) + STROKEclean["HospitalDiagnosis"].astype(str)
STROKEclean["Merged_Impression"] = STROKEclean["PrimaryImpression"].fillna('') + STROKEclean["SecondaryImpression"].fillna('')

##drop reduntant columns
#STROKE.drop(["EmergencyDepartmentDiagnosis", "HospitalDiagnosis"], axis =1,  inplace =True)

##View Header and count null

print("\n header:\n",STROKEclean.head(2))

print("\n count null after merge:\n", STROKEclean.isna().sum())

In [ ]:
#STROKEclean.dtypes
print("\n count NaN after merge:\n",STROKEclean[STROKEclean == 'nannan'].count())
#STROKEclean['overall_diagnosis'].eq('nannan').sum()##df1[['overall_diagnosis ', 'col3', 'col4']].eq(1).sum()

In [ ]:
#get the unique values
#PI = pd.DataFrame(STROKEclean["PrimaryImpression"].unique())
#PI
# STROKEclean['Uniques'] = pd.DataFrame(STROKEclean["SecondaryImpression"].unique())
# #SI[0] = SI['Uniques']
# #SI.drop([0])
# #SI.describe()
# STROKEclean.dtypes
# STROKEclean.head()
#df.drop(columns=['B', 'C'])
#STROKEclean.drop(['Uniques'], axis=1)
#Write as .csv
#SI.to_excel("E:\DAEN_690\Data\SI.xlsx",sheet_name='STROKE' , index=False)

STROKEclean["Merged_Impression"].unique() 
# I63

In [ ]:
#THe distinct impressions related to strokes are:
## Primary Impression / ##Secondary Impression

#['Neuro - Stroke/CVA (I63.9)',
#'Intracranial - Stroke (CVA) Hemorrhagic (I62.9)',
#in combination with
#,'Environment - Heat Exhaustion/Stroke (T67.5)',
# 'Environment - Heatstroke (T67.0)']



In [ ]:
## create a new column that is 1 if the overall_diagnosis column contains the string "A41.9", 0 if not
#https://www.icd10data.com/ICD10CM/Codes/I00-I99/I60-I69/I63-/I63.9#:~:text=Cerebral%20infarction%2C%20unspecified,-2016%202017%202018&text=Billable%2FSpecific%20Code-,I63.,effective%20on%20October%201%2C%202021.
#ICD-10 Categories I60-I69 Cerebrovascular Disease: #https://providers.bcbsal.org/portal/documents/10226/306297/Correctly+Coding+Cerebrovascular+Disease/87166c14-7be2-4728-b32b-95ec8f802fea

#in the data set I62.9 & I63.9 are the only distinct codes used for Stroke.
STROKEclean["STROKE_outcome"] = pd.np.where(STROKEclean["overall_diagnosis"].str.contains("I6", na=False, case=False), "1", "0")#)#)

#pd.np.where(STROKEclean["overall_diagnosis"].str.contains("STROKE", na=False, case=False), "1", 
                                #pd.np.where(STROKEclean["overall_diagnosis"].str.contains("STROKE", na=False), "1",

    
    
print(STROKEclean.head())
#Write as .csv
STROKEclean.to_excel("E:\DAEN_690\Data\PdmSTROKE.xlsx",sheet_name='STROKE' , index=False)

In [ ]:
## create a new column that is Sepsis if the PrimaryImpression column contains the string "Sepsis", Other if not

STROKEclean["Overall_Impression"] = pd.np.where(STROKEclean["Merged_Impression"].str.contains("I6", na=False, case=False), 1, 0)#)
                               
                             
        #pd.np.where(STROKEclean["PrimaryImpression"].str.contains("STROKE", na=False, case=False), "STROKE", 
print(STROKEclean.head())
#Write as .csv
STROKEclean.to_excel("E:\DAEN_690\Data\STROKEclean.xlsx",sheet_name='STROKE' , index=False)

In [ ]:
print("\n PImpressions Values: \n", STROKEclean['Merged_Impression'].unique())
print("\n PImpr Values: \n", STROKEclean['Overall_Impression'].unique())


In [ ]:
##Drop redundant column 
STROKEclean.drop(['MonthYear','PrimaryImpression','SecondaryImpression', "EmergencyDepartmentDiagnosis","HospitalDiagnosis","overall_diagnosis"], axis =1,  inplace =True)
STROKEclean.head()

In [ ]:
STROKEclean.columns

Merge with Patient Demographic Information

In [ ]:
# merging the files how='inner' by default.
PdmSTROKE = pd.merge(PatDemo,STROKE, on='PtIDInternal') 
print(PdmSTROKE.head())
#Write as .csv
PdmSTROKE.to_excel("E:\DAEN_690\Data\PdmSTROKE.xlsx",sheet_name='STROKE' , index=False)


In [ ]:
import altair as alt
#from vega_datasets import data

source = PdmSTROKE

PatSTROKEChart = alt.Chart(source).mark_bar().encode(
    x='AgeGroup_HIPAA:O',
    y='count(AgeGroup_HIPAA):Q',
    color='PrimaryImpr:N',
    column='StrokeStatus:N',
    tooltip=['PrimaryImpr', 'STROKE_outcome', 'AgeGroup_HIPAA', 'count(AgeGroup_HIPAA)']
).interactive()
PatSTROKEChart .display()
#PatDemChart.save('PatDemChart.html')

#Final Cleaning of the DataSets

In [ ]:
## Filter the dataframes to only keep rows where "isOutcomeFacility" = 1
## this is because it doesn't make sense to count these rows which all have NULL for diagnosis when doing analysis
# df.loc[df['column_name'] == some_value]
PdmSEPSIS = PdmSEPSIS.loc[PdmSEPSIS['isOutcomeFacility'] == 1]
PdmSTEMI = PdmSTEMI.loc[PdmSTEMI['isOutcomeFacility'] == 1]
PdmSTROKE = PdmSTROKE.loc[PdmSTROKE['isOutcomeFacility'] == 1]

## Filter the dataframes to get rid of rows where overall_diagnosis is not null
PdmSEPSIS = PdmSEPSIS[PdmSEPSIS['overall_diagnosis'].str.len() > 0]
PdmSTEMI = PdmSTEMI[PdmSTEMI['overall_diagnosis'].str.len() > 0]
PdmSTROKE = PdmSTROKE[PdmSTROKE['overall_diagnosis'].str.len() > 0]

# df = df[df['str_field'].str.len() > 0]

sepsis_demo.head(10)

# Finding out the common patients across all data sets.

In [ ]:
# merging the files how='inner' by default.
#PdmMerged = pd.merge(PatDemo,SEPSIS,STEMI,STROKE, on='PtIDInternal') 
PdmMerged = PatDemo.merge(SEPSIS, on='PtIDInternal').merge(STEMI,on='PtIDInternal').merge(STROKE,on='PtIDInternal')
PdmMerged 

#Alternatively
#df4 = pd.merge(pd.merge(PatDemo,SEPSIS,on='PtIDInternal'),pd.merge(STEMI,STROKE,on='PtIDInternal'))
# print(df4)
# #Write as .csv
# df4.to_excel("E:\DAEN_690\Data\RepeatingPat.xlsx",sheet_name='RepeatingPat' , index=False)

In [ ]:
# import altair as alt
# from vega_datasets import data

# chart = alt.Chart(data.cars.url).mark_point().encode(
#     x='Horsepower:Q',
#     y='Miles_per_Gallon:Q',
#     color='Origin:N'
# )

# chart.save('chart.html')

In [ ]:
 #Anaconda Prompt-jupyter notebook list-for token

#Analytics & Algorithms/ EDA

SEPSIS

In [ ]:
#https://medium.com/@szabo.bibor/how-to-create-a-seaborn-correlation-heatmap-in-python-834c0686b88e
#Checking the correlations among attributes.

plt.figure(figsize=(20,20))
sns.heatmap(PdmSEPSIS.corr(),cbar=True,annot=True,cmap='YlGnBu') #vmin=0.3, vmax=0.99,

In [ ]:
#https://stackoverflow.com/questions/61956336/find-high-correlations-in-a-large-coefficient-matrix
#https://stackoverflow.com/questions/67711552/python-split-pandas-dataframe-by-range-of-values

coeff = PdmSEPSIS.corr()

# 0.3 is used for illustration 
# replace with your actual value
thresh_low = 0.3
thresh_high = 1

#mask = coeff.abs().lt(thresh)
# or mask = coeff> thresh
mask = (coeff> thresh_low) & (coeff < thresh_high) 

coeff.where(mask).stack()

# #Remove Diagonol
# corr = PdmSEPSIS.corr()
# import numpy as np
# np.fill_diagonal(corr.values, np.nan)
# print(corr)

In [ ]:
#Plot only the Correlations between 0.3 and 1.0

CorCoeff = coeff.where(mask)

plt.figure(figsize=(20,20))
sns.heatmap(CorCoeff,cbar=True,annot=True,cmap='YlGnBu') #.reshape(26,1)
plt.title('Correlations between 0.3 and 1.0')

In [ ]:
#Heat map of Nans
plt.figure(figsize=(20,10))
sns.heatmap(CorCoeff.isnull(), cbar=False, cmap="YlGnBu")
plt.show()

In [ ]:
#Lets drop irrelavant  columns:
PdmSEPSIS.drop(['Accuracy','Triage'],axis =1,  inplace =True)
print(PdmSEPSIS.head(3))
print("\n info: ", PdmSEPSIS.info())

Prepare Data Set for Training

In [ ]:
#Now you have to take off the Response column into y
import pandas as pd     # manipulate dataframe
import seaborn as sns   # visualization of classification

from sklearn.model_selection import train_test_split   # to split the data
from sklearn.linear_model import LogisticRegression    # to bring logistic regression
from sklearn.tree import DecisionTreeClassifier        # to bring decision tree
from sklearn.ensemble import RandomForestClassifier    # to bring random forest
from sklearn.model_selection import GridSearchCV       # to find best hyper parameters

from sklearn import metrics                            # to create confusion matrix

X = PdmSEPSIS.drop('sepsis_outcome',axis=1)
y = PdmSEPSIS['sepsis_outcome']

In [ ]:
print ("\n Xhead: ",X.head(3))
print ("\n yhead: ",y.head(3))       

In [ ]:
#And then we are going to split this X and y into train and test data.
#https://towardsdatascience.com/why-do-we-set-a-random-state-in-machine-learning-models-bb2dc68d8431

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1357)

# Logistic Regression & Multinomial Logistic Regression


In [ ]:
#https://www.analyticsvidhya.com/blog/2017/08/skilltest-logistic-regression/
#Yes, we can apply logistic regression on 3 classification problem, We can use One Vs all method for 3 class classification in logistic regression.
#https://machinelearningmastery.com/multinomial-logistic-regression-with-python/

LR = LogisticRegression()    # Bring empty logistic regression model

In [ ]:
#Logistic regression uses maximum likely hood estimate for training a logistic regression.

LR.fit(x_train,y_train)      # Train the model with training data

In [ ]:
y_pred_LR = LR.predict(x_test)    # Get predicted y from the logistic regression model

In [ ]:
# Get the classification accuracy, Recall, and Presicion with the metrics function.
LR_accuracy = metrics.accuracy_score(y_pred_LR,y_test)
print('Classification accuracy = ',LR_accuracy)
print('Recall = ',metrics.recall_score(y_test, y_pred_LR, average='weighted')) #average='macro' #average='micro' #average='weighted'
print('Precision = ',metrics.precision_score(y_test, y_pred_LR, average='weighted')) #average='macro' #average='micro' #average='weighted'

In [ ]:
#Check the class lable distribution
ytrain = pd.DataFrame(y_train)

Class_Distribution = pd.DataFrame(ytrain['sepsis_outcome'].value_counts())
Class_Distribution['Percentages'] = (Class_Distribution['sepsis_outcome']/Class_Distribution['sepsis_outcome'].sum())*100
Class_Distribution



# Multinomial Logistic Regression

In [ ]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
# define dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
X, y = x_train, y_train
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

logistic regression model with default penalty achieved a mean classification accuracy of about 91 percent

calling the predict() function to make a prediction for new data

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# clf = RandomForestRegressor()
# clf.fit(X,y)

##https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# # Load the diabetes dataset
# diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# # Use only one feature
# diabetes_X = diabetes_X[:, np.newaxis, 2]

# # Split the data into training/testing sets
# diabetes_X_train = diabetes_X[:-20]
# diabetes_X_test = diabetes_X[-20:]

# # Split the targets into training/testing sets
# diabetes_y_train = diabetes_y[:-20]
# diabetes_y_test = diabetes_y[-20:]

# # Create linear regression object
# regr = linear_model.LinearRegression()

# # Train the model using the training sets
# regr.fit(diabetes_X_train, diabetes_y_train)

# # Make predictions using the testing set
# diabetes_y_pred = regr.predict(diabetes_X_test)

# # The coefficients
# print("Coefficients: \n", regr.coef_)
# # The mean squared error
# print("Mean squared error: %.2f" % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# # The coefficient of determination: 1 is perfect prediction
# print("Coefficient of determination: %.2f" % r2_score(diabetes_y_test, diabetes_y_pred))

# # Plot outputs
# plt.scatter(diabetes_X_test, diabetes_y_test, color="black")
# plt.plot(diabetes_X_test, diabetes_y_pred, color="blue", linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

In [ ]:
#type(x_test)

# make a prediction with a multinomial logistic regression model
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
# define dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
X, y = x_train, y_train
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# fit the model on the whole dataset
model.fit(X, y)
# define a single row of input data
##row = [1.89149379, -0.39847585, 1.63856893, 0.01647165, 1.51892395, -3.52651223, 1.80998823, 0.58810926, -0.02542177, -0.52835426]
row = x_test
## predict the class label
#yhat = model.predict([row])
yhatt = model.predict(row)
## summarize the predicted class
#print('Predicted Class: %d' % yhat[0])
print("\n yhat: ", yhatt[0])

# The coefficients
print("\n Coefficients: \n", model.coef_)

# The mean squared error
#print("Mean squared error: %.2f" % mean_squared_error(y_test, yhat)) #could not convert string to float: 'UnderTriage'
# The coefficient of determination: 1 is perfect prediction
#print("Coefficient of determination: %.2f" % r2_score(y_test, yhat)) #ValueError: could not convert string to float: 'UnderTriage'

### Plot outputs
# plt.scatter(x_test, y_test, color="black")
# plt.plot(x_test, yhat, color="blue", linewidth=3)
# plt.xticks(())
# plt.yticks(())
# plt.show() #ValueError: x and y must be the same size


In [ ]:
#type(yhat)
#yhat.shape #(633,)
#yhat.dtype
#yhat[1]
np.where(yhatt == 0) #Check for the indexes
#np.where(yhatt == 'UnderTriage')
#np.where(yhatt == 'OverTriage')

In [ ]:
# ### predict a multinomial probability distribution
yhat = model.predict_proba(row)
# summarize the predicted probabilities
print('\n Predicted Probabilities: %s' % yhat[0])

In [ ]:
#Class:        Predicted Probabilities:
        
#1 :         Predicted Probabilities: [0.00303091 0.99696909]
#0:          Predicted Probabilities: [0.00342926 0.99657074]

# Triage:        Predicted Probabilities: [0.00330011 0.99221901 0.00448088]

# OverTriage:    Predicted Probabilities: [0.9918642  0.00597873 0.00215706]

# UnderTriage:   Predicted Probabilities: [9.15067953e-04 2.14479714e-03 9.96940135e-01]  


In [ ]:
#https://machinelearningmastery.com/multinomial-logistic-regression-with-python/
# tune regularization for multinomial logistic regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot


# get the dataset
def get_dataset():
    #X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1, n_classes=3)
    X, y = x_train, y_train
    return X, y

# get a list of models to evaluate
def get_models():
    models = dict()
    for p in [0.0, 0.0001, 0.001, 0.01, 0.1, 1.0]:
        # create name for model
        key = '%.4f' % p
        # turn off penalty in some cases
        if p == 0.0:
            # no penalty in this case
            models[key] = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='none')
        else:
            models[key] = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=p)
    return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 
# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model and collect the scores
    scores = evaluate_model(model, X, y)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize progress along the way
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
  

In [ ]:
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.title('L2 Penalty Configuration vs. Accuracy for Multinomial Logistic Regression')
pyplot.show()  


#larger penalty we use on this dataset- the smaller the C value - the worse the performance of the model.
#L2 penalty with weighting values in the range from 0.0001 to 1.0 on a log scale
#no penalty or 0.0
C value 1.0 has the better accuracy score of 100% than no penalty of 0.00


# Support Vector Machine

In [ ]:
#https://towardsdatascience.com/support-vector-machines-explained-with-python-examples-cb65e8172c85
#SVM 

#Import Library
from sklearn import svm
import numpy as np


X=x_train

y=y_train

model = svm.SVC() 
model.fit(X, y)
model.score(X, y)

In [ ]:
#predict 
print("\n predicted:\n\n", model.predict(x_test))
# get support vectors
print("\n support_vectors:\n\n",model.support_vectors_)
# get indices of support vectors
print("\n support:\n\n",model.support_)

In [ ]:
##Multi-class classification
#https://www.baeldung.com/cs/svm-multiclass-classification#:~:text=SVM%20Multiclass%20Classification%20in%20Python&text=We%20developed%20two%20different%20classifiers,on%20the%20same%20data%20set.
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score



#create two different classifiers, Polynomial kernel, and another one with RBF kernel
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(x_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(x_train, y_train)

#calculate the efficiency of the two models

poly_pred = poly.predict(x_test)
rbf_pred = rbf.predict(x_test)

#calculate the accuracy and f1 scores for SVM with Polynomial kernel

poly_accuracy = accuracy_score(y_test, poly_pred) #percentage of the true positive and true negative to all data points
poly_f1 = f1_score(y_test, poly_pred, average='weighted') #harmonic mean between precision and recall, and both depend on the false positive and false negative
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

#accuracy and f1 scores for SVM with RBF kernel:
rbf_accuracy = accuracy_score(y_test, rbf_pred) #percentage of the true positive and true negative to all data points
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted') #harmonic mean between precision and recall, and both depend on the false positive and false negative
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

#SVM hyperparameters, like C, gamma, and degree 
#f1 score when the data set isn’t balanced



In [ ]:
# get number of support vectors for each class
model.n_support_

# DECISION TREE

In [ ]:
DTC = DecisionTreeClassifier()    # Bring empty decision tree model
                                  # you can set the name whatever you want. it doesn't need to be "DTC"
    
DTC.fit(x_train,y_train)          # Train the decision tree model with training data

y_pred_DTC = DTC.predict(x_test)  # Get predicted y from the decision tree model

Decision_accuracy = metrics.accuracy_score(y_pred_DTC,y_test)  # Get the classification accuracy
print(Decision_accuracy)

GRID SEARCH METHOD

In [ ]:
grid_DTC = DecisionTreeClassifier()    # Bring empty decision tree

# Prepare the combination of parameters.
param_dist={"criterion":["gini","entropy"],       # 2 options for criterion
            'max_depth': range(2,8),              # 6 options for max_depth
            'max_leaf_nodes': range(50,150,50)}   # 2 options for max_leaf_nodes
# So, the total number of combination is 2*6*2 = 24

# cv means cross validation and this concept will come in the future
grid_DTC = GridSearchCV(grid_DTC, param_grid= param_dist, cv=5, n_jobs=-1)   
#Number of jobs to run in parallel.  -1 means using all processors. 

grid_DTC.fit(x_train, y_train)     # Fit all of the 24 combinations


best_parameters = grid_DTC.best_params_   # Get best parameters 
print(best_parameters)

# Create a decision tree with best parameters and train it with whole train data
after_grid_DTC = DecisionTreeClassifier(criterion= 'entropy', max_depth= 6, max_leaf_nodes= 50, random_state= 1357)
after_grid_DTC = after_grid_DTC.fit(x_train, y_train)

y_pred_grid = after_grid_DTC.predict(x_test)     # Get predicted y from the decision tree model

Decision_accuracy = metrics.accuracy_score(y_pred_grid,y_test)
print(Decision_accuracy)

# RANDOM FOREST

In [ ]:
RF = RandomForestClassifier(100,n_jobs=-1,random_state=1357)   # Bring empty random forest model, 100 is the number of trees in the RF
RF.fit(x_train,y_train)   # Train the model with training data

y_pred_RF = RF.predict(x_test)   # Get predicted y from the random forest model

# Get the classification accuracy, Recall, and Presicion with the metrics function.
print('Accucary = ',metrics.accuracy_score(y_pred_RF,y_test))
print('Recall = ',metrics.recall_score(y_test, y_pred_RF, average='weighted'))#average='macro' #average='micro' #average='weighted'
print('Precision = ',metrics.precision_score(y_test, y_pred_RF, average='weighted'))#average='macro' #average='micro' #average='weighted'

Visualize the confusion metrics

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred_RF)    # Create a confusion matrix from your predicted y and original y value.
                                                    # In this code, we use the result of random forest, but you can replace with another classifier

sns.heatmap(cm,annot=True,cmap='Reds',fmt='.0f')    # Bring heatmap from the seaborn and fit our confusion matrix
                                                    # annot = True can shows the confusion matrix values
                                                    # camp can decide the color of heatmap
                                                    # fmt can decide the type of presentation of confusion matrix values

ROC curve

In [ ]:
# With false_positive_rate and true_positive_rate, we can plot the ROC curve.

# To get false_positive_rate and true_positive_rate, we need to use predicted probability and actual labels.

# y_pred_proba = RF.predict_proba(x_test)   # Get the probability of classification from the Random Forest model
#                                           # In this code, we use the result of random forest, but you can replace with another classifier

# false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_test, y_pred_proba[:,1])     
# # Orignally predict_proba return 2 values for both class. 
# # So, we picked second one which give the probability of class 1 #ValueError: multiclass format is not supported

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline  
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.title('ROC')
# plt.plot(false_positive_rate, true_positive_rate)  
# # By simply ploting false positive rate and true positive rate, you can plot the roc curve

In [ ]:
#Area Under the Curve
# auc = metrics.auc(false_positive_rate, true_positive_rate)   # metrics also privide the auc area
# print(auc)

# Vizualizations:

Demography distributions:

In [ ]:
#Age group distribution among different demographies

import altair as alt


source = PatDemo

PtDemchart1 = alt.Chart(source).mark_bar().encode(
    x='AgeGroup_HIPAA:O',
    y='count(AgeGroup_HIPAA):Q',
    color='PatientEthnicity:N'
)


# PtDemchart2 = alt.Chart(PatDemo).mark_bar().encode(
#      y='AgeGroup_HIPAA',
#      x='count(AgeGroup_HIPAA)',
#     color='PatientRace:N'
#  )

PtDemchart3 = alt.Chart(PatDemo).mark_bar().encode(
     x='AgeGroup_HIPAA:O',
     y='count(AgeGroup_HIPAA):Q',
    color='PatientSex:N'
 )


PtDemchart1 | PtDemchart3#PtDemchart2|



In [ ]:
#interactive brush selection charts
import altair as alt
from vega_datasets import data
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')
#pip install altair_data_server
#alt.data_transformers.enable('data_server')

source = PatDemo
brush = alt.selection(type='interval')

alt.Chart(source).mark_point().encode(
    x='AgeGroup_HIPAA:O',
    y='count(AgeGroup_HIPAA):Q',
    color=alt.condition(brush, 'PatientRace:O', alt.value('grey')),
).add_selection(brush)

In [ ]:
#interactive Charts

import altair as alt
from vega_datasets import data

source = PatDemo
selection = alt.selection_multi(fields=['PatientSex'], bind='legend')
PatDemChart = alt.Chart(source).mark_bar().encode(
    x='AgeGroup_HIPAA:O',
    y='count(AgeGroup_HIPAA):Q',
    color='PatientSex:N',
    column='PatientRace:N',
    tooltip=['PatientEthnicity', 'PatientSex', 'AgeGroup_HIPAA', 'count(AgeGroup_HIPAA)']
).add_selection(
    selection
)
PatDemChart .display()
#PatDemChart.save('PatDemChart.html')

In [ ]:
#Interactive Agegroup distribution among Patient Race and Gender #https://altair-viz.github.io/gallery/streamgraph.html
import altair as alt
from vega_datasets import data

#source = pd.read_csv("E:\DAEN_690\Data\us-employment.csv")

source = PatDemo
selection = alt.selection_multi(fields=['PatientRace'], bind='legend')

PatDemInteractive= alt.Chart(source).mark_area().encode(
    alt.X('AgeGroup_HIPAA:O', axis=alt.Axis(domain=False, tickSize=0)),
    alt.Y('count(AgeGroup_HIPAA):Q', stack='center', axis=None),
    alt.Color('PatientRace:N', scale=alt.Scale(scheme='category20b')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['PatientSex:N','PatientRace:N', 'AgeGroup_HIPAA:O', 'count(AgeGroup_HIPAA):Q']
  
).add_selection(
    selection
).add_selection(brush).interactive()

#PatDemInteractive.display()
#PatDemInteractive.save('chart.html', embed_options={'renderer':'svg'})
PatDemInteractive.display()
#chart.save('chart.png', scale_factor=2.0) #Size Resolutions

#PatDemInteractive.save('chart.html')

In [ ]:
#Second viz
source = PatDemo
selection = alt.selection_multi(fields=['PatientRace'], bind='legend')

PatDemInteractive1= alt.Chart(source).mark_area().encode(
    alt.X('AgeGroup_HIPAA:O', axis=alt.Axis(domain=False, tickSize=0)),
    alt.Y('count(AgeGroup_HIPAA):Q', stack='center', axis=None),
    alt.Color('PatientEthnicity:N', scale=alt.Scale(scheme='category20b')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['PatientSex','PatientEthnicity', 'AgeGroup_HIPAA', 'count(AgeGroup_HIPAA)']
  
).add_selection(
    selection
).add_selection(brush).interactive()

PatDemInteractive1.display()

In [ ]:
#Age group among gender distribution

import altair as alt
from vega_datasets import data #Only to use data from vegas data sets or else can be ignored

source = PatDemo

Agegroup = alt.Chart(source).mark_circle(size=60).encode(
    x='AgeGroup_HIPAA:O',
    y='count(AgeGroup_HIPAA):Q',
    color='PatientSex:N',
    tooltip=['PatientRace:N', 'PatientSex:N', 'AgeGroup_HIPAA:O', 'count(AgeGroup_HIPAA):Q']
).interactive()
Agegroup.save('Agegroup.html')
Agegroup

In [ ]:
#https://towardsdatascience.com/dash-for-beginners-create-interactive-python-dashboards-338bfcb6ffa4
#Plotly Dash
#pip install dash --Anaconda Prompt
#pip install jupyter-dash
# ! pip install dash-html-components                                         
# ! pip install dash-core-components                                     
# ! pip install plotly
# from dash import Dash, html, dcc, Input, Output
# import pandas as pd
# import plotly.express as px
#https://plotly.com/python/dot-plots/
#import dash package

import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
import plotly.express as px

df = PdmSEPSIS 

fig = px.scatter(df, y="ProviderImpression", x="sepsis_outcome", color="Triage", symbol = "Triage")
fig.update_traces(marker_size=10)
fig.show()

In [ ]:
import altair as alt
from vega_datasets import data

source = PdmSEPSIS

alt.Chart(source).mark_circle(size=60).encode(
    x='ProviderImpression',
    y='Triage',
    color='SepsisStatus_Alert',
    tooltip=['PatientSex_Female', 'SepsisStatus_Alert', 'ProviderImpression', 'Triage']
).interactive()

In [ ]:
import altair as alt
from vega_datasets import data

source = PdmSEPSIS

alt.Chart(source).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Triage:N'
).properties(
    width=150,
    height=150
).repeat(
    row=['SepsisStatus_Alert', 'ProviderImpression', 'Accuracy'],
    column=['Accuracy', 'ProviderImpression', 'SepsisStatus_Alert']
).interactive()

In [ ]:
import altair as alt
import pandas as pd
import numpy as np



source = PdmSEPSIS

base = alt.Chart(source).mark_circle(opacity=0.5).transform_fold(
    fold=['Accuracy', 'ProviderImpression', 'SepsisStatus_Alert'],
    as_=['category', 'y']
).encode(
    alt.X('Triage:N'),
    alt.Y('y:Q'),
    alt.Color('category:N')
)

base + base.transform_loess('Triage', 'y', groupby=['category']).mark_line(size=4)

In [ ]:
import altair as alt
from vega_datasets import data

source = PdmSEPSIS

alt.Chart(source, title="The FFS Performance").mark_point().encode(
    alt.X(
        'ProviderImpression:N',
        title="ProviderImpression",
        scale=alt.Scale(zero=False),
        axis=alt.Axis(grid=False)
    ),
    alt.Y(
        'Triage:N',
        title="",
        sort='-x',
        axis=alt.Axis(grid=True)
    ),
    color=alt.Color('sepsis_outcome:N', legend=alt.Legend(title="sepsis_outcome")),
    row=alt.Row(
        'SepsisStatus_Alert:N',
        title="",
        sort=alt.EncodingSortField(field='ProviderImpression', op='sum', order='descending'),
    )
).properties(
    height=alt.Step(20)
).configure_view(stroke="transparent")

In [ ]:
#How does Providor Impression affects the Outcome
plt.figure(figsize=(10,10))
plt.scatter(x='ProviderImpression',y='sepsis_outcome',data=PdmSEPSIS)
plt.xlabel('ProviderImpression')
plt.ylabel('sepsis_outcome')

# #How does Providor Impression affects the Triage
# plt.figure(figsize=(10,10))
# plt.scatter(x='ProviderImpression',y='Triage',data=PdmSEPSIS)
# plt.xlabel('ProviderImpression')
# plt.ylabel('Triage')

In [ ]:
#Count Nulls
print(PdmSEPSIS.isnull().sum())

In [ ]:
#With the help of heatmap, we can see the amount of data that is missing from the attribute
sns.heatmap(PdmSEPSIS.isnull(),cbar=False,cmap='viridis')

In [ ]:
# PdmSEPSIS.groupby(['AgeGroup_HIPAA','Accuracy'])['Accuracy'].count().unstack().plot(legend=True)
# plt.title('AgeGroup_HIPAA and Accuracy')
# plt.xlabel('AgeGroup_HIPAA')
# plt.ylabel('Accuracy')
# plt.show()

In [ ]:
#Multi-panel Scatter Plot with Linked Brushing
source = PdmSEPSIS

selection = alt.selection_multi(fields=['Accuracy'], bind='legend')


Accuracy = alt.Chart(source, title ='Accuracy of Primary Impressions among Age Groups and Gender', width=400, height=300).mark_circle().encode(
    
    x = 'AgeGroup_HIPAA:N',
    y = alt.Y('count(AgeGroup_HIPAA):Q',title='Total Population'),   

    color=alt.condition(brush, 'Accuracy:N', alt.value('lightgray')),      
 
    tooltip=['PatientSex','Accuracy', 'AgeGroup_HIPAA', 'count(AgeGroup_HIPAA)'],
 
    facet='PatientSex',
      
    size = alt.Size('count(AgeGroup_HIPAA)',title = 'Total Populaion',scale=alt.Scale(range=[100, 500]))
).add_selection(
    selection
).add_selection(brush)
Accuracy.display()

In [ ]:
#Chart for reference

source = PdmSEPSIS
#.mark_line()/.mark_bar()/.mark_point()
#scales = alt.selection_interval(bind='scales')  
# interval = alt.selection_interval()
# make_example(interval)
Accuracy = alt.Chart(source, title ='Accuracy of Primary Impressions among Age Groups and Gender', width=400, height=300).mark_circle().encode(
    
    x = 'AgeGroup_HIPAA:N',
    y = alt.Y('count(AgeGroup_HIPAA):Q',title='Total Population'),   
#   alt.X('AgeGroup_HIPAA:N', axis=alt.Axis(domain=False, tickSize=0)),
#   alt.Y('count(AgeGroup_HIPAA):Q',title='Total Population',stack='center', axis=None),
    color=alt.condition(brush, 'Accuracy:N', alt.value('lightgray')),      
 
    tooltip=['PatientSex','Accuracy', 'AgeGroup_HIPAA', 'count(AgeGroup_HIPAA)'],
    #opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    facet='PatientSex',
      
    size = alt.Size('count(AgeGroup_HIPAA)',title = 'Total Populaion',scale=alt.Scale(range=[100, 500]))
    ).add_selection(
    selection
    ).add_selection(brush).interactive()
Accuracy.display()

In [ ]:
import altair as alt
from vega_datasets import data

source = PdmSEPSIS
selection = alt.selection_multi(fields=['Accuracy'], bind='legend')
PatSEPSISChart = alt.Chart(source).mark_bar().encode(
    x='AgeGroup_HIPAA:O',
    y='count(AgeGroup_HIPAA):Q',
    color='Triage:N',
    column='SepsisStatus:N',
    tooltip=['Triage', 'sepsis_outcome', 'AgeGroup_HIPAA', 'count(AgeGroup_HIPAA)']
).add_selection(
    selection
    ).interactive()
PatSEPSISChart .display()
#PatDemChart.save('PatDemChart.html')